In [1]:
pip install unstructured

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     - ----------------------------------- 30.7/981.5 kB 187.9 kB/s eta 0:00:06
     - ----------------------------------- 41.0/981.5 kB 196.9 kB/s eta 0:00:05
     - ----------------------------------- 41.0/981.5 kB 196.9 kB/s eta 0:00:05
     - ----------------------------------- 41.0/981.5 kB 196.9 kB/s eta 0:00:05
     -- ---------------------------------- 61.4/981.5 kB 172.4 kB/s eta 0:00:06
     --- --------------------------------- 81.9/981.5 kB 176.6 kB/s eta 0:00:06
     --- --------------------------------- 81.9/981.5 kB 176.6 kB/s eta 0:00:06
 

In [2]:
pip -q install chromadb  langchain tiktoken

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show chromadb

Name: chromadb
Version: 0.4.24
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: c:\users\gnana\anaconda3\envs\mchatbot\lib\site-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, graphlib-backport, grpcio, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pulsar-client, pydantic, pypika, PyYAML, requests, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [5]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import Pinecone
#from pinecone import Pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [7]:
import langchain_community.document_loaders.unstructured
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

In [8]:
loader=PyPDFDirectoryLoader("data")

In [10]:
extracted_data=loader.load()

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks= text_splitter.split_documents(extracted_data)

In [12]:
print(len(text_chunks))

7020


In [13]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install -U sentence-transformers

   ---------------------------------------- 0.0/163.3 kB ? eta -:--:--
   -- ------------------------------------- 10.2/163.3 kB ? eta -:--:--
   ------- ------------------------------- 30.7/163.3 kB 325.1 kB/s eta 0:00:01
   -------------- ------------------------ 61.4/163.3 kB 465.5 kB/s eta 0:00:01
   ---------------------- ---------------- 92.2/163.3 kB 581.0 kB/s eta 0:00:01
   ---------------------------- --------- 122.9/163.3 kB 599.1 kB/s eta 0:00:01
   -------------------------------------- 163.3/163.3 kB 575.5 kB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.6.0
    Uninstalling sentence-transformers-2.6.0:
      Successfully uninstalled sentence-transformers-2.6.0
Note: you may need to restart the kernel to use updated packages.


In [15]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [16]:
embeddings= download_hugging_face_embeddings()

c:\Users\gnana\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\gnana\anaconda3\envs\mchatbot\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gnana\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In

In [17]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
persist_directory='db'

In [19]:
vectordb = Chroma.from_documents(documents=text_chunks,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [20]:
vectordb.persist()

In [21]:
vectordb=None

In [22]:
vectordb = Chroma(persist_directory=persist_directory,embedding_function=embeddings)

In [23]:
vectordb

In [43]:
retriever= vectordb.as_retriever(search_kwargs={"k": 2})

In [44]:
docs= retriever.get_relevant_documents("Acne ?")

In [45]:
docs

[Document(page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25', metadata={'page': 38, 'source': 'data\\Medical_book.pdf'}),
 Document(page_content='ent purposes. For example, lotions, soaps, gels, andcreams containing benzoyl peroxide or tretinoin may beused to clear up mild to moderately severe acne.Isotretinoin (Accutane) is prescribed only for verysevere, disfiguring acne.\nAcne is a skin condition that occurs when pores or', metadata={'page': 239, 'source': 'data\\Medical_book.pdf'})]

In [46]:
len(docs)

2

In [47]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [48]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [37]:
pip install ctransformers


   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB 435.7 kB/s eta 0:00:23
   ---------------------------------------- 0.1/9.9 MB 465.5 kB/s eta 0:00:22
   ---------------------------------------- 0.1/9.9 MB 595.3 kB/s eta 0:00:17
    --------------------------------------- 0.2/9.9 MB 610.0 kB/s eta 0:00:16
    --------------------------------------- 0.2/9.9 MB 655.1 kB/s eta 0:00:15
    --------------------------------------- 0.2/9.9 MB 654.9 kB/s eta 0:00:15
   - -------------------------------------- 0.3/9.9 MB 679.3 kB/s eta 0:00:15
   - -------------------------------------- 0.3/9.9 MB 698.7 kB/s eta 0:00:14
   - -------------------------------------- 0.4/9.9 MB 692.6 kB/s eta 0:00:14
   - -------------------------------------- 0.4/9.9 MB 689.7 kB/s eta 0:00:14
   - -------------------------------------- 0.5/9.9 MB 704.5 kB/s eta 0:00:14
   - -------------------------------------- 0.5/9.9 MB 699.5 kB/s eta 0:

In [49]:
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers
#from ctransformers import AutoModelForCausalLM

In [51]:
llm=CTransformers(model="models/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [52]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [53]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\gnana\anaconda3\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Twins are formed when a single fertilized egg splits in two and develops into two separate embryos. This process is called monozygotic twinning or dizygotic twinning, depending on whether the split occurs before or after the embryo has developed its own identity (or differentiated).
